## Financial Database 

### TODO: 
1. Build a database of end-of-day price data for a set of companies from the same market sector (to start with). Eventually extend to multiple market sectors.
2. Write a function that calculates time-series correlation between two stocks. Eventually write other functions that let you interpret useful things about the market (or call functions from rami's stat predictor model)
3. Build a company "profile": a set of important parameters about a company that give an idea of its valuation, and that an investor might be interested in
4. Come up with a way to search for "financial situations" about companies using their time-series information and other data you could obtain. For example: "how long was AAPL's volatility high after the 2008 market crash?" "what are the most correlated stocks in the technology sector?"
5. Make it possible to obtain any kind of high-level statement from a combination of other statements. You could use some kind of parsing API (like the one used by wolfram alpha, if there are any open-source ones) or create your own interpreter, using simple commands. Up till this point it is kind of like a financial wolfram alpha.
6. Make the process automatic: write an algorithm that can detect an interesting pattern among all this higher-level data that you could create, and could notify you instantly. For example: "the fundamentals of JP Morgan are very similar to those before the 2008 market crash, beware."

## How it could be used

The user could directly query situations they would be interested in, or other components such as the interactive graph could call it. The optimizer could call it to obtain information about a stock's volatility, returns, stability, etc.

The end objective is to capture what the market is doing today, rather than try to predict it. These tools try to approximate the work of an analyst going through the numbers and making conclusions.

In [ ]:
# /Users/mbpr/Desktop/AUB/2018-2019/Spring 2018-2019/FYP/QuantInvest/db/db.json

# REST API token:
# pk_a91586fc254847e08b030f86765dabaf

In [2]:
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import get_sector_performance
from iexfinance.refdata import get_symbols
from datetime import datetime

import functions as fn
import numpy as np
import pandas as pd
import json
import importlib
import requests

In [9]:
# to re-import functions file after modification
importlib.reload(fn)

<module 'functions' from '/Users/mbpr/Desktop/AUB/2018-2019/Spring 2018-2019/FYP/QuantInvest/db/functions.py'>

In [10]:
start = datetime(2018, 2, 24)
end = datetime(2019, 2, 24)

##### TODO:
- Build a table for each company containing relevant raw data. For now just include closing price. Later expand. 
- Store these tables in a json file.
- Write interfaces to the file so it can update automatically.
- Store information about companies in a workbook-type data structure. Each company's info is in one sheet, and there are many sheets. Sectors can be also grouped into meta-workbooks etc.
- Write useful functions to extract high-level information from the data in the file.

In [11]:
symbols = get_symbols()

In [12]:
print(fn.getSymbol(0, symbols, "Microsoft Corporation"))

MSFT


In [13]:
aapl = get_historical_data('AAPL', start, end, output_format = 'pandas')
goog = get_historical_data('GOOG', start, end, output_format = 'pandas')
msft = get_historical_data('MSFT', start, end, output_format = 'pandas')

In [45]:
col1 = aapl['close']
col2 = goog['close']
col3 = msft['close']

# make a 3d table to store all the companies' data, each in a 2d dataframe
col = pd.concat([col1, col2, col3], 1)

In [47]:
# duct tape solution until i figure out multiIndex
col.columns = ['Apple', 'Google', 'Microsoft']

In [12]:
# store the df in a pickle. later store the multiIndex locally, eventually use Nader's database
col.to_pickle('/Users/mbpr/Desktop/AUB/2018-2019/Spring 2018-2019/FYP/QuantInvest/db/db.json')

In [13]:
# read the pickle
col = pd.read_pickle('db.json')

In [79]:
# answer the question: which stocks were most correlated in June 2018? automate this
corr1 = fn.rollingCorr(0, col1, col2, 5)
corr2 = fn.rollingCorr(0, col2, col3, 5)
corr3 = fn.rollingCorr(0, col1, col3, 5)

avg1 = np.average(corr1[corr1.notnull()])
avg2 = np.average(corr2[corr1.notnull()])
avg3 = np.average(corr3[corr1.notnull()])

np.argmax([avg1, avg2, avg3])

1

##### CONSOLE

In [3]:
stocks = requests.get('https://cloud.iexapis.com/beta/stock/aapl/financials?token=pk_a91586fc254847e08b030f86765dabaf')

In [33]:
stocks.json()

{'symbol': 'AAPL',
 'balancesheet': [{'reportDate': '2018-12-31',
   'currentCash': 44771000000,
   'shortTermInvestments': 41656000000,
   'receivables': 18077000000,
   'inventory': 4988000000,
   'otherCurrentAssets': 12432000000,
   'currentAssets': 140828000000,
   'longTermInvestments': 158608000000,
   'propertyPlantEquipment': 39597000000,
   'goodwill': 0,
   'intangibleAssets': None,
   'otherAssets': 34686000000,
   'totalAssets': 373719000000,
   'accountsPayable': 44293000000,
   'currentLongTermDebt': 9772000000,
   'otherCurrentLiabilities': 42249000000,
   'totalCurrentLiabilities': 108283000000,
   'longTermDebt': 92989000000,
   'otherLiabilities': 23607000000,
   'minorityInterest': 0,
   'totalLiabilities': 255827000000,
   'commonStock': 40970000000,
   'retainedEarnings': 80510000000,
   'treasuryStock': None,
   'capitalSurplus': None,
   'shareholderEquity': 117892000000,
   'netTangibleAssets': 117892000000}]}